# Introduction

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
In this competition, we'll forecast twelve-hours of traffic flow in a major U.S. metropolitan area. Time, space, and directional features give us the chance to model interactions across a network of roadways.
</div>

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from datetime import datetime

import warnings
import time
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

RANDOM_STATE = 18
FOLDS = 5

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')

# Exploring Train Data

<div class = "alert alert-info" role = "alert" style="font-size:14px; font-family:verdana;">

📌 <b><u>Observations in Train Data</u></b><br>

* There are <b><u>6</u></b> columns: <b><u>1</u></b> row_id, <b><u>1</u></b> datetime, <b><u>3</u></b> categorical, and <b><u>1</u></b> continuous<br>   (x, y are considered as categorical as only few values available in each column)<br>
* There are total of <b><u>848,834</u></b> rows in train dataset<br>
* <b><u>"congestion"</u></b> is the our target variable, which is a continuous variable with range from <b><u>0</u></b> to <b><u>100</u></b><br>
* There are no missing / null values in this dataset<br>
* This is a <b><u>time series</u></b> analysis
    
</div>

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check Train data
</div>

In [ ]:
train.head()

In [ ]:
print(f'\033[92mNumber of rows in the table: {train.shape[0]}')
print(f'\033[94mNumber of columns in the table: {train.shape[1]}')
print(f'\033[91mNumber of observations in the table: {train.count().sum()}')
print(f'\033[91mNumber of missing values in the table: {sum(train.isnull().sum())}')
print(f'\033[91mNumber of duplicated records: {train.duplicated().sum()}')
print()
print(f'\033[95mData types')
print(f'\033[90m{train.dtypes}')
print()
print(f'\033[95mData type counts')
print(f'\033[90m{train.dtypes.value_counts()}')
print()
print(f'\033[95mUnique value in each column')
print(f'\033[90m{train.nunique()}')

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check null values
</div>

In [ ]:
train.isnull().sum()

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Quick view on continuous columns
</div>

In [ ]:
train.describe()

# Exploring Test Data

<div class = "alert alert-info" role = "alert" style="font-size:14px; font-family:verdana;">

📌 <b><u>Observations in Test Data</u></b><br>

* There are <b><u>6</u></b> columns: <b><u>1</u></b> row_id, <b><u>1</u></b> datetime, <b><u>3</u></b> categorical, and <b><u>1</u></b> continuous<br>
* There are total of <b><u>2,340</u></b> rows in test dataset, which is around just <b><u>0.27%</u></b> of Train Data<br>
* There are no missing / null values in this dataset
    
</div>

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check test data
</div>

In [ ]:
test.head()

In [ ]:
print(f'\033[92mNumber of rows in the table: {test.shape[0]}')
print(f'\033[94mNumber of columns in the table: {test.shape[1]}')
print(f'\033[91mNumber of observations in the table: {test.count().sum()}')
print(f'\033[91mNumber of missing values in the table: {sum(test.isnull().sum())}')
print(f'\033[91mNumber of duplicated records: {test.duplicated().sum()}')
print()
print(f'\033[95mData types')
print(f'\033[90m{test.dtypes}')
print()
print(f'\033[95mData type counts')
print(f'\033[90m{test.dtypes.value_counts()}')
print()
print(f'\033[95mUnique value in each column')
print(f'\033[90m{test.nunique()}')

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check null values
</div>

In [ ]:
test.isnull().sum()

In [ ]:
print("Test set rows / Train set rows = " + str(round(test.count()[0] / train.count()[0] * 100, 2)))

# Basic Exploratory Data Analysis

<div class = "alert alert-info" role = "alert" style="font-size:14px; font-family:verdana;">

🔍 <b><u>Check List</u></b><br>

* Distriution of categorical variabless between train and test set<br>
    - Seems to be quite similar<br>
  <br>
* Relationship between categorical variables and target variable<br>
    - Some directions have lower average congestion value compare with others<br>
  <br>
* Relationship between date and target variable<br>
    - There seems to be a trend<br>
    
</div>

In [ ]:
train['time'] = pd.to_datetime(train['time'], format='%Y-%m-%d %H:%M:%S')
test['time'] = pd.to_datetime(test['time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
sns.set_style('whitegrid')
fig = plt.figure(figsize = (12, 8))

# x distribution
ax1 = fig.add_subplot(3, 2, 1)

sns.countplot(x = 'x', data = train, palette="Set2")
plt.xticks()

ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)

ax1.set_xlabel("x value", fontsize=14, labelpad=10)
ax1.set_ylabel("Count", fontsize=14, labelpad=10)
ax1.set_title('Distribution of x - Train Set', loc = 'center', fontsize = 14, fontweight = 'bold')

ax2 = fig.add_subplot(3, 2, 2)

sns.countplot(x = 'x', data = test, palette="Set2")
plt.xticks()

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)

ax2.set_xlabel("x value", fontsize=14, labelpad=10)
ax2.set_ylabel("Count", fontsize=14, labelpad=10)
ax2.set_title('Distribution of x - Test Set', loc = 'center', fontsize = 14, fontweight = 'bold')

# y distribution
ax3 = fig.add_subplot(3, 2, 3)

sns.countplot(x = 'y', data = train, palette="Set2")
plt.xticks()

ax3.spines['right'].set_visible(False)
ax3.spines['top'].set_visible(False)

ax3.set_xlabel("y value", fontsize=14, labelpad=10)
ax3.set_ylabel("Count", fontsize=14, labelpad=10)
ax3.set_title('Distribution of y - Train Set', loc = 'center', fontsize = 14, fontweight = 'bold')

ax4 = fig.add_subplot(3, 2, 4)

sns.countplot(x = 'y', data = test, palette="Set2")
plt.xticks()

ax4.spines['right'].set_visible(False)
ax4.spines['top'].set_visible(False)

ax4.set_xlabel("y value", fontsize=14, labelpad=10)
ax4.set_ylabel("Count", fontsize=14, labelpad=10)
ax4.set_title('Distribution of y - Test Set', loc = 'center', fontsize = 14, fontweight = 'bold')

# direction distribution
ax5 = fig.add_subplot(3, 2, 5)

sns.countplot(x = 'direction', data = train, palette="Set2")
plt.xticks()

ax5.spines['right'].set_visible(False)
ax5.spines['top'].set_visible(False)

ax5.set_xlabel("direction", fontsize=14, labelpad=10)
ax5.set_ylabel("Count", fontsize=14, labelpad=10)
ax5.set_title('Distribution of direction - Train Set', loc = 'center', fontsize = 14, fontweight = 'bold')

ax6 = fig.add_subplot(3, 2, 6)

sns.countplot(x = 'direction', data = test, palette="Set2")
plt.xticks()

ax6.spines['right'].set_visible(False)
ax6.spines['top'].set_visible(False)

ax6.set_xlabel("direction", fontsize=14, labelpad=10)
ax6.set_ylabel("Count", fontsize=14, labelpad=10)
ax6.set_title('Distribution of direction - Test Set', loc = 'center', fontsize = 14, fontweight = 'bold')
fig.tight_layout()

In [ ]:
categorical_columns = train[['x', 'y', 'direction']].columns.to_numpy()

In [ ]:
fig = plt.figure(figsize = (10, 9))
rows = 3
cols = 1

for idx, categorical_column in enumerate(categorical_columns):
  ax = fig.add_subplot(rows, cols, idx + 1)
  sns.barplot(x = categorical_column, y = 'congestion', data = train.groupby(categorical_column).mean('congestion')['congestion'].reset_index(), palette = 'Set2')
  
  ax.xaxis.set_tick_params(labelsize=10, size=0, pad=5)
  ax.yaxis.set_tick_params(labelsize=10, size=0, pad=5)

  ax.spines['right'].set_visible(False)
  ax.spines['top'].set_visible(False)
    
  ax.set_ylabel('congestion')

  ax.set_xlabel(categorical_column + " value")
  ax.bar_label(ax.containers[0])
  
  ax.set_title('Average congestion by ' + categorical_column, loc = 'center', fontsize = 14, fontweight = 'bold', pad = 20)

  ax.legend()

fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure(figsize = (12, 3))

# date
ax = fig.add_subplot(1, 1, 1)
sns.lineplot(x = train['time'].dt.date, y = 'congestion', data = train)

ax.xaxis.set_tick_params(labelsize=10, size=0, pad=5)
ax.yaxis.set_tick_params(labelsize=10, size=0, pad=5)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_ylabel('congestion')

ax.set_xlabel('date')

ax.set_title('Average congestion by date', loc = 'center', fontsize = 14, fontweight = 'bold', pad = 20)

fig.tight_layout()
fig.show()

# Feature Engineering

<div class = "alert alert-info" role = "alert" style="font-size:14px; font-family:verdana;">

🔍 <b><u>Some Ideas & Hypothesis</u></b><br>

* <u><b>Week of day</b></u> might have impact on congestion<br>
    - Yes, weekend have lower average congestion values<br>
    <br>
* <u><b>Busy hours</b></u> might have impact on congestion<br>
    - Yes, from noon to 8 pm. the average congestion values are above 50<br>
    <br>
* <u><b>Holidays</b></u> might have impact on congestion
    - Yes, congestion values are lower on <b><u>Memorial's day</u></b>, <b><u>July 4th</u></b>, and <b><u>Labor day</u></b><br>
    <br>
* x, y, and direction might have <u><b>combined effect</b></u> on congestion<br>

    
</div>

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Let's create some new features based on <b><u>time</u></b> column
</div>

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start='1991-01-01', end='1991-12-01')

In [ ]:
print('US Holidays in 1991')
holidays

In [ ]:
dir_mapper = {'EB': [1,0], 
              'NB': [0,1], 
              'SB': [0,-1], 
              'WB': [-1,0], 
              'NE': [1,1], 
              'SW': [-1,-1], 
              'NW': [-1,1], 
              'SE': [1,-1]}

In [ ]:
def feature_engineering(data):   
    data['month'] = data['time'].dt.month
    data['day'] = data['time'].dt.day
    data['time_part'] = data['time'].dt.time
    data['hour'] = data['time'].dt.hour
    data['minute'] = data['time'].dt.minute
    data['weekday'] = data['time'].dt.strftime("%A")
    data['x_y'] = data['x'].map(str) + '-' + data['y'].map(str)
    data['x_y_direction'] = data['x'].map(str) + '-' + data['y'].map(str) + '-' + data['direction']
    data['direction_hour'] = data['direction'] + '-' + data['time'].dt.hour.map(str)
    data['converted_direction_coord_0'] = data['direction'].map(lambda x: dir_mapper[x][0])
    data['converted_direction_coord_1'] = data['direction'].map(lambda x: dir_mapper[x][1])
    data['holiday'] = np.where(data['time'].isin(holidays), 'Yes', 'No')
    return data

In [ ]:
for data in [train, test]:
    data = feature_engineering(data)

In [ ]:
train[train['holiday'] == 'Yes'][['time', 'weekday', 'holiday']].drop_duplicates()

In [ ]:
train.head()

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check differences between days of the week
</div>

In [ ]:
fig = plt.figure(figsize = (12, 6))

# boxplot
ax1 = fig.add_subplot(1, 2, 1)

sns.boxplot(x = 'weekday', y = 'congestion', data = train, palette = 'Set2')

ax1.xaxis.set_tick_params(labelsize=10, size=0, pad=5)
ax1.yaxis.set_tick_params(labelsize=10, size=0, pad=5)

ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)

ax1.set_ylabel('congestion')

ax1.set_xlabel('Weekday', labelpad = 10)

ax1.set_title('Average congestion by weekday', loc = 'center', fontsize = 14, fontweight = 'bold', pad = 20)

ax1.legend()

# barplot
ax2 = fig.add_subplot(1, 2, 2)

sns.barplot(x = 'weekday', y = 'congestion', data = train, palette = 'Set2')

ax2.xaxis.set_tick_params(labelsize=10, size=0, pad=5)
ax2.yaxis.set_tick_params(labelsize=10, size=0, pad=5)

ax2.spines['right'].set_visible(False)
ax2.spines['top'].set_visible(False)

ax2.set_ylabel('congestion')

ax2.set_xlabel('Weekday', labelpad = 10)

ax2.set_title('Average congestion by weekday', loc = 'center', fontsize = 14, fontweight = 'bold', pad = 20)
ax2.bar_label(ax2.containers[0])
ax2.legend()

fig.tight_layout()
fig.show()

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check congestion by time
</div>

In [ ]:
temp = train.groupby('time_part').mean('congestion')['congestion'].reset_index()
temp['busy'] = temp['congestion'] >= 50

fig = plt.figure(figsize = (12, 20))

# time
ax = fig.add_subplot(2, 1, 2)
sns.barplot(x = 'congestion', y = 'time_part', hue = 'busy', data = temp, palette = 'Set2')

ax.xaxis.set_tick_params(labelsize=10, size=0, pad=5)
ax.yaxis.set_tick_params(labelsize=10, size=0, pad=5)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.axvline(50, ls = '--', lw = 1, color = 'black')

ax.set_ylabel('congestion')

ax.set_xlabel('time_part')

ax.set_title('Average congestion by time', loc = 'center', fontsize = 14, fontweight = 'bold', pad = 20)

fig.tight_layout()
fig.show()

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check impact on holidays
</div>

In [ ]:
fig = plt.figure(figsize = (8, 4))

# boxplot
ax1 = fig.add_subplot(1, 1, 1)

sns.barplot(x = 'weekday', y = 'congestion', hue = 'holiday', data = train[~train['weekday'].isin(['Saturday', 'Sunday'])], palette = 'Set2')

ax1.xaxis.set_tick_params(labelsize=10, size=0, pad=5)
ax1.yaxis.set_tick_params(labelsize=10, size=0, pad=5)

ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)

ax1.set_ylabel('congestion')

ax1.set_xlabel('Weekday', labelpad = 10)

ax1.set_title('Average congestion by weekday', loc = 'center', fontsize = 14, fontweight = 'bold', pad = 20)

ax1.legend(title = 'Holiday', loc = 'upper right')

fig.tight_layout()
fig.show()

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Let's drop records on holidays as they could be outliers in the data
</div>

In [ ]:
train = train[train['holiday'] == 'No'].drop(['holiday'], axis = 1)
test = test.drop(['holiday'], axis = 1)

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Check combined effect
</div>

In [ ]:
sns.set_style('whitegrid')
fig = plt.figure(figsize = (8, 4))

# x distribution
ax = fig.add_subplot(1, 1, 1)

sns.barplot(x = 'x_y', y = 'congestion', data = train, palette="Set2")
plt.xticks()

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xlabel("x_y value", fontsize=14, labelpad=10)
ax.set_ylabel("Count", fontsize=14, labelpad=10)
ax.set_title('Combined effect on x and y', loc = 'center', fontsize = 14, fontweight = 'bold')

fig.tight_layout()

# Model Building

In [ ]:
train.drop(['row_id', 'time'], axis = 1, inplace = True)
test.drop(['row_id', 'time'], axis = 1, inplace = True)

In [ ]:
TARGET = 'congestion'
FEATURES = [col for col in train.columns if col not in ['row_id', TARGET]]

In [ ]:
enc = OrdinalEncoder()
enc.fit(train[FEATURES])
train[FEATURES] = enc.transform(train[FEATURES]).astype(int)
test[FEATURES] = enc.transform(test[FEATURES]).astype(int)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
catb_params = {
    'objective': 'MAE',
    'cat_features': FEATURES,
    'task_type': 'GPU'
}

In [ ]:
catb_predictions = []
catb_mae = []
catb_r2 = []
catb_fimp = []


cv = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(cv.split(train[FEATURES], train[TARGET])):
    
    print(10*"=", f"Fold = {fold + 1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][FEATURES], train.iloc[valid_idx][FEATURES]
    y_train, y_valid = train[TARGET].iloc[train_idx], train[TARGET].iloc[valid_idx]
    
    model = CatBoostRegressor(**catb_params)
    model.fit(X_train, y_train, verbose = 0)
    
    pred_valid = model.predict(X_valid)
    
    mae = mean_absolute_error(y_valid, pred_valid)
    catb_mae.append(mae)
    
    r2 = r2_score(y_valid, pred_valid)
    catb_r2.append(r2)
    
    run_time = time.time() - start_time
    
    print(f"Fold = {fold + 1}, MAE: {mae:.2f}, R2: {r2:.2f}, Run Time: {run_time:.2f}s")
    test_pred = model.predict(test[FEATURES])
    fim = pd.DataFrame(index = FEATURES,
                      data = model.feature_importances_,
                      columns = [f'{fold}_importance'])
    catb_fimp.append(fim)
    catb_predictions.append(test_pred)
    
print("Mean MAE :", np.mean(catb_mae), "Mean R2 :", np.mean(catb_r2))

In [ ]:
catb_fis_df = pd.concat(catb_fimp, axis = 1).head(15)
catb_fis_df.sort_values('1_importance').plot(kind = 'barh', figsize = (15, 10), title = 'Feature Importance Across Folds')
plt.legend(loc='lower right')
plt.show()

In [ ]:
catb_submission = submission.copy()[['row_id']]
catb_submission["congestion"] = np.clip(np.round(np.squeeze(np.mean(np.column_stack(catb_predictions),axis = 1)), 0), 0, 100).astype(int)
catb_submission.to_csv("catb.csv",index=False)
catb_submission.head()

In [ ]:
xgb_params = {  
    'n_estimators': 1000,
    'tree_method': 'gpu_hist',    
    'predictor': 'gpu_predictor', 
    'booster': 'gbtree',         
    'eta': 0.3,                   
    'gamma': 0,                   
    'max_depth': 6,               
    'lambda': 1,                  
    'alpha': 0,                  
}

In [ ]:
xgb_predictions = []
xgb_mae = []
xgb_r2 = []
xgb_fimp = []


cv = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(cv.split(train[FEATURES], train[TARGET])):
    
    print(10*"=", f"Fold = {fold + 1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][FEATURES], train.iloc[valid_idx][FEATURES]
    y_train, y_valid = train[TARGET].iloc[train_idx], train[TARGET].iloc[valid_idx]
    
    model = XGBRegressor(**xgb_params)
    model.fit(X_train, y_train, verbose = 0)
    
    pred_valid = model.predict(X_valid)
    
    mae = mean_absolute_error(y_valid, pred_valid)
    xgb_mae.append(mae)
    
    r2 = r2_score(y_valid, pred_valid)
    xgb_r2.append(r2)
    
    run_time = time.time() - start_time
    
    print(f"Fold = {fold + 1}, MAE: {mae:.2f}, R2: {r2:.2f}, Run Time: {run_time:.2f}s")
    test_pred = model.predict(test[FEATURES])
    fim = pd.DataFrame(index = FEATURES,
                      data = model.feature_importances_,
                      columns = [f'{fold}_importance'])
    xgb_fimp.append(fim)
    xgb_predictions.append(test_pred)
    
print("Mean MAE :", np.mean(xgb_mae), "Mean R2 :", np.mean(xgb_r2))

In [ ]:
xgb_fis_df = pd.concat(xgb_fimp, axis = 1).head(15)
xgb_fis_df.sort_values('1_importance').plot(kind = 'barh', figsize = (15, 10), title = 'Feature Importance Across Folds')
plt.legend(loc='lower right')
plt.show()

In [ ]:
xgb_submission = submission.copy()[['row_id']]
xgb_submission["congestion"] = np.clip(np.round(np.squeeze(np.mean(np.column_stack(xgb_predictions),axis = 1)), 0), 0, 100).astype(int)
xgb_submission.to_csv("xgb.csv",index=False)
xgb_submission.head()

In [ ]:
lgb_params = {
    'objective': 'mean_absolute_error',    
    'metric': 'mae',    
    'device': 'gpu',              
    'n_estimators': 1000,        
    'learning_rate': 0.1,        
    'num_leaves': 31,            
    'max_depth': -1,              
    'bagging_freq': 0,            
    'feature_fraction': 1,        
    'lambda_l1': 0,               
    'lambda_l2': 0,              
}

In [ ]:
lgb_predictions = []
lgb_mae = []
lgb_r2 = []
lgb_fimp = []


cv = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(cv.split(train[FEATURES], train[TARGET])):
    
    print(10*"=", f"Fold = {fold + 1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][FEATURES], train.iloc[valid_idx][FEATURES]
    y_train, y_valid = train[TARGET].iloc[train_idx], train[TARGET].iloc[valid_idx]
    
    model = LGBMRegressor(**lgb_params)
    model.fit(X_train, y_train, verbose = 0)
    
    pred_valid = model.predict(X_valid)
    
    mae = mean_absolute_error(y_valid, pred_valid)
    lgb_mae.append(mae)
    
    r2 = r2_score(y_valid, pred_valid)
    lgb_r2.append(r2)
    
    run_time = time.time() - start_time
    
    print(f"Fold = {fold + 1}, MAE: {mae:.2f}, R2: {r2:.2f}, Run Time: {run_time:.2f}s")
    test_pred = model.predict(test[FEATURES])
    fim = pd.DataFrame(index = FEATURES,
                      data = model.feature_importances_,
                      columns = [f'{fold}_importance'])
    lgb_fimp.append(fim)
    lgb_predictions.append(test_pred)
    
print("Mean MAE :", np.mean(lgb_mae), "Mean R2 :", np.mean(lgb_r2))

In [ ]:
lgb_fis_df = pd.concat(lgb_fimp, axis = 1).head(15)
lgb_fis_df.sort_values('1_importance').plot(kind = 'barh', figsize = (15, 10), title = 'Feature Importance Across Folds')
plt.legend(loc='lower right')
plt.show()

In [ ]:
lgb_submission = submission.copy()[['row_id']]
lgb_submission["congestion"] = np.clip(np.round(np.squeeze(np.mean(np.column_stack(lgb_predictions),axis = 1)), 0), 0, 100).astype(int)
lgb_submission.to_csv("lgb.csv",index=False)
lgb_submission.head()

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
Average of 3 models
</div>

In [ ]:
mean_submission = submission.copy()
pred_mean = np.clip(np.round(np.squeeze(np.mean(np.column_stack(xgb_predictions + lgb_predictions + catb_predictions),axis = 1)), 0), 0, 100).astype(int)
mean_submission["target"] = pred_mean
mean_submission.to_csv("pred_mean.csv",index=False)
mean_submission.head()

In [ ]:
'''
X_train, X_valid, y_train, y_valid = train_test_split(train[FEATURES].values, train[TARGET].values, test_size = 0.2, random_state=5)

cv = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = RANDOM_STATE)

model = CatBoostRegressor(objective = 'RMSE', cat_features = FEATURES, task_type = 'GPU')

grid = {
    'iterations': np.arange(10, 500, 10),
    'depth': np.arange(2, 16, 2),
    'learning_rate': [0.01, 0.05, 0.1]
        }

model_random = RandomizedSearchCV(estimator = model, param_distributions = grid, n_iter = 5, 
                                  cv = cv, verbose=1, random_state=RANDOM_STATE)
model_random.fit(train[FEATURES], train[TARGET])
'''

In [ ]:
'''
print(model_random.cv_results_)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", model_random.best_estimator_)
print("\n The best score across ALL searched params:\n", model_random.best_score_)
print("\n The best parameters across ALL searched params:\n", model_random.best_params_)
'''

In [ ]:
'''
model_random.predict(test[FEATURES])
'''

In [ ]:
'''
catb_grid_submission = submission.copy()[['row_id']]
catb_grid_submission["congestion"] = np.clip(np.round(model_random.predict(test[FEATURES]), 0), 0, 100).astype(int)
catb_grid_submission.to_csv("catb_grid.csv",index=False)
catb_grid_submission.head()
'''

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
What if we train model using only Monday data and exclude holidays?
</div>

In [ ]:
# NEW_FEATURES = [i for i in FEATURES if i not in ['weekday', 'holiday']]

In [ ]:
# train[(train['weekday'] == 1) & (train['holiday'] == 0)][NEW_FEATURES]

In [ ]:
#catb_params = {
#    'objective': 'MAE',
#    'cat_features': NEW_FEATURES,
#    'task_type': 'GPU'
#}

In [ ]:
'''
catb_monday_predictions = []
catb_monday_mae = []
catb_monday_r2 = []
catb_monday_fimp = []


cv = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(cv.split(train[(train['weekday'] == 1) & (train['holiday'] == 0)][NEW_FEATURES], train[(train['weekday'] == 1) & (train['holiday'] == 0)][TARGET])):
    
    print(10*"=", f"Fold = {fold + 1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train[(train['weekday'] == 1) & (train['holiday'] == 0)].iloc[train_idx][NEW_FEATURES], train[(train['weekday'] == 1) & (train['holiday'] == 0)].iloc[valid_idx][NEW_FEATURES]
    y_train, y_valid = train[(train['weekday'] == 1) & (train['holiday'] == 0)][TARGET].iloc[train_idx], train[(train['weekday'] == 1) & (train['holiday'] == 0)][TARGET].iloc[valid_idx]
    
    model = CatBoostRegressor(**catb_params)
    model.fit(X_train, y_train, verbose = 0)
    
    pred_valid = model.predict(X_valid)
    
    mae = mean_absolute_error(y_valid, pred_valid)
    catb_monday_mae.append(mae)
    
    r2 = r2_score(y_valid, pred_valid)
    catb_monday_r2.append(r2)
    
    run_time = time.time() - start_time
    
    print(f"Fold = {fold + 1}, MAE: {mae:.2f}, R2: {r2:.2f}, Run Time: {run_time:.2f}s")
    test_pred = model.predict(test[NEW_FEATURES])
    fim = pd.DataFrame(index = NEW_FEATURES,
                      data = model.feature_importances_,
                      columns = [f'{fold}_importance'])
    catb_monday_fimp.append(fim)
    catb_monday_predictions.append(test_pred)
    
print("Mean MAE :", np.mean(catb_monday_mae), "Mean R2 :", np.mean(catb_monday_r2))
'''

In [ ]:
'''
catb_fis_df = pd.concat(catb_monday_fimp, axis = 1).head(10)
catb_fis_df.sort_values('1_importance').plot(kind = 'barh', figsize = (15, 10), title = 'Feature Importance Across Folds')
plt.show()
'''

In [ ]:
'''
catb_monday_submission = submission.copy()[['row_id']]
catb_monday_submission["congestion"] = np.clip(np.round(np.squeeze(np.mean(np.column_stack(catb_monday_predictions),axis = 1)), 0), 0, 100).astype(int)
catb_monday_submission.to_csv("catb_monday.csv",index=False)
catb_monday_submission.head()
'''

<div style="font-size:16px; color:#4169E1; font-family:verdana;">
And what if we train model using only Monday data in <b>August</b> and <b>September</b> (exclude holidays)?
</div>

In [ ]:
'''
catb_monday_month_predictions = []
catb_monday_month_mae = []
catb_monday_month_r2 = []
catb_monday_month_fimp = []


cv = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(cv.split(train[(train['weekday'] == 1) & (train['holiday'] == 0) & (train['month'] >= 4)][NEW_FEATURES], train[(train['weekday'] == 1) & (train['holiday'] == 0) & (train['month'] >= 4)][TARGET])):
    
    print(10*"=", f"Fold = {fold + 1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train[(train['weekday'] == 1) & (train['holiday'] == 0) & (train['month'] >= 4)].iloc[train_idx][NEW_FEATURES], train[(train['weekday'] == 1) & (train['holiday'] == 0) & (train['month'] >= 4)].iloc[valid_idx][NEW_FEATURES]
    y_train, y_valid = train[(train['weekday'] == 1) & (train['holiday'] == 0) & (train['month'] >= 4)][TARGET].iloc[train_idx], train[(train['weekday'] == 1) & (train['holiday'] == 0 & (train['month'] >= 4))][TARGET].iloc[valid_idx]
    
    model = CatBoostRegressor(**catb_params)
    model.fit(X_train, y_train, verbose = 0)
    
    pred_valid = model.predict(X_valid)
    
    mae = mean_absolute_error(y_valid, pred_valid)
    catb_monday_month_mae.append(mae)
    
    r2 = r2_score(y_valid, pred_valid)
    catb_monday_month_r2.append(r2)
    
    run_time = time.time() - start_time
    
    print(f"Fold = {fold + 1}, MAE: {mae:.2f}, R2: {r2:.2f}, Run Time: {run_time:.2f}s")
    test_pred = model.predict(test[NEW_FEATURES])
    fim = pd.DataFrame(index = NEW_FEATURES,
                      data = model.feature_importances_,
                      columns = [f'{fold}_importance'])
    catb_monday_month_fimp.append(fim)
    catb_monday_month_predictions.append(test_pred)
    
print("Mean MAE :", np.mean(catb_monday_month_mae), "Mean R2 :", np.mean(catb_monday_month_r2))
'''

In [ ]:
'''
catb_fis_df = pd.concat(catb_monday_month_fimp, axis = 1).head(10)
catb_fis_df.sort_values('1_importance').plot(kind = 'barh', figsize = (15, 10), title = 'Feature Importance Across Folds')
plt.show()
'''

In [ ]:
'''
catb_monday_month_submission = submission.copy()[['row_id']]
catb_monday_month_submission["congestion"] = np.clip(np.round(np.squeeze(np.mean(np.column_stack(catb_monday_month_predictions),axis = 1)), 0), 0, 100).astype(int)
catb_monday_month_submission.to_csv("catb_monday_month.csv",index=False)
catb_monday_month_submission.head()
'''

<div class = "alert alert-info" role = "alert" style="font-size:14px; font-family:verdana;">

📌 <b><u>Conslusion</u></b><br>

* Use less data did not increase model performance
    
</div>